In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
import spotipy
from spotipy.oauth2 import SpotifyOAuth

CLIENT_ID=os.getenv("CLIENT_ID")
CLIENT_SECRET=os.getenv("CLIENT_SECRET")
REDIRECT_URI="http://localhost:5566"
SCOPE = "user-library-read user-read-currently-playing streaming user-read-playback-state user-read-recently-played"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=CLIENT_ID,
                                               client_secret=CLIENT_SECRET,
                                               redirect_uri=REDIRECT_URI,
                                               scope=SCOPE))

In [2]:
saved_tracks = sp.current_user_saved_tracks(50)
for idx, item in enumerate(saved_tracks['items']):
    track = item['track']
    print(idx, track['artists'][0]['name'], " – ", track['name'])

0 Tom Misch  –  Crazy Dream
1 Acid Pauli  –  Ahmed
2 Ariel Pink  –  Baby
3 KOKOROKO  –  Abusey Junction
4 Khruangbin  –  Dearest Alfred (MyJoy)
5 Los Tucanes De Tijuana  –  El Tucanazo
6 SZA  –  Good Days
7 Leykenda  –  Anxiety - Edit
8 Paige  –  Losing Control
9 Nicolas Jaar  –  Mi Mujer
10 Kasper Bjørke  –  Heaven - Nicolas Jaar Remix
11 LJ MASE  –  Memories
12 Sido  –  Schono ke
13 BHZ  –  Nelly
14 JACKBOYS  –  GATTI
15 Drake  –  Time Flies
16 Johnny Nash  –  I Can See Clearly Now - Edit
17 Curtis Harding  –  On And On
18 slowthai  –  feel away (feat. James Blake & Mount Kimbie)
19 slowthai  –  nhs
20 slowthai  –  MAZZA (feat. A$AP Rocky)
21 Bausa  –  Centre Court
22 Miksu / Macloud  –  Lonely
23 Luciano  –  SUICIDE DOORS
24 Ufo361  –  7
25 Samra  –  Lost (feat. TOPIC42)
26 Benal  –  Tænker Lidt På dig (feat. Wads.png)
27 Apache 207  –  Angst
28 James Brown  –  The Payback
29 RÜFÜS DU SOL  –  Innerbloom
30 Queen  –  Don't Stop Me Now - Remastered 2011
31 Citybois  –  Sig Mig
32 Jay 

In [33]:
last_50_tracks_played = sp.current_user_recently_played(50)
for idx, item in enumerate(last_50_tracks_played['items']):
    track = item['track']
    # print(idx, track['artists'][0]['name'], " – ", track['name'])

# interesting meta data
played_at = last_50_tracks_played['items'][0]['played_at'] # time it was played
album_name = last_50_tracks_played['items'][0]['track']['album']['name'] # album name
album_href = last_50_tracks_played['items'][0]['track']['album']['href'] # use as subject?
album_id = last_50_tracks_played['items'][0]['track']['album']['id'] # use as subject?
album_images = last_50_tracks_played['items'][0]['track']['album']['images'] # [Array] for interface
album_release_date = last_50_tracks_played['items'][0]['track']['album']['release_date']
artists = last_50_tracks_played['items'][0]['track']['artists'] # Array
first_artist_href = last_50_tracks_played['items'][0]['track']['artists'][0]['href']
first_artist_id = last_50_tracks_played['items'][0]['track']['artists'][0]['id']
first_artist_name = last_50_tracks_played['items'][0]['track']['artists'][0]['name']
name = last_50_tracks_played['items'][0]['track']['name']
popularity = last_50_tracks_played['items'][0]['track']['popularity']
href = last_50_tracks_played['items'][0]['track']['href']
duration_ms = last_50_tracks_played['items'][0]['track']['duration_ms']

In [57]:
import rdflib
g = rdflib.Graph()

# parse in an RDF file hosted on the Internet
result = g.parse("http://localhost:3000")

# loop through each triple in the graph (subj, pred, obj)
for subj, pred, obj in g:
    # check if there is at least one triple in the Graph
    if (subj, pred, obj) not in g:
       raise Exception("It better be!")

# print the number of "triples" in the Graph
print("graph has {} statements.".format(len(g)))
# prints graph has 86 statements.

# print out the entire Graph in the RDF Turtle format
print(g.serialize(format="turtle").decode("utf-8"))

graph has 11 statements.
@prefix dc: <http://purl.org/dc/terms/> .
@prefix ldp: <http://www.w3.org/ns/ldp#> .
@prefix posix: <http://www.w3.org/ns/posix/stat#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://localhost:3000> a ldp:BasicContainer,
        ldp:Container,
        ldp:Resource ;
    dc:modified "2021-02-04T20:39:42.168000+00:00"^^xsd:dateTime ;
    ldp:contains <http://localhost:3000/.acl> ;
    posix:mtime 1612471182 ;
    posix:size 96 .

<http://localhost:3000/.acl> a ldp:Resource ;
    dc:modified "2021-02-04T20:39:42.169000+00:00"^^xsd:dateTime ;
    posix:mtime 1612471182 ;
    posix:size 390 .


